In [141]:
import numpy as np
import pandas as pd

In [142]:
num_samples = 10000
latent_dim = 256
batch_size = 64  
input_texts = []
target_texts = []
epochs = 100  
input_characters = set()
target_characters = set()

In [143]:
with open("C:\MyWork\MyLearning\Career Growth\ML\Files\DataSet\spa.txt", "r", encoding="utf-8") as f:
    lines = f.read().split("\n")

In [144]:
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split("\t")
    
    ############### A ###############
    target_text = "\t" + target_text + "\n"
    input_texts.append(input_text)
    target_texts.append(target_text)
    ############### B ###############
    
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)
            
print(input_characters)

print("\n")

print(target_characters)

{'9', 'x', 'V', 'L', 'I', '0', '4', 'e', ':', 'l', 'p', 'g', 'E', 'i', 'O', 'H', 'v', '3', 'U', 'F', 'y', '7', 'u', 'm', 'K', 'C', 'P', 'j', 's', 'N', 'k', 'o', 'r', 'd', 'G', '1', 'D', 'J', '5', 'Q', '2', 'a', 'c', 'B', 'M', 't', 'S', '-', 'w', '6', 'T', ' ', ',', '8', 'q', '$', 'n', 'b', 'W', "'", 'Y', '!', '.', 'A', 'f', '?', 'z', 'R', 'h'}


{'x', '¿', 'I', '0', 'p', 'O', 'ñ', 'y', 'c', 't', '-', 'í', '6', ',', 'W', 'Y', '!', '.', 'A', 'h', '4', 'e', 'E', '«', 'Ú', 'U', '»', 'u', 'j', 'N', 'k', '\t', '1', 'D', 'J', 'a', 'M', "'", 'á', '?', 'z', 'é', 'V', 'L', '"', 'ü', ':', 'l', 'g', '7', 'F', 's', 'o', 'd', 'ó', 'Á', '¡', 'q', 'f', 'Ó', 'n', 'i', 'H', 'v', '3', 'm', 'C', 'P', 'r', 'G', '5', 'Q', '2', 'B', 'S', 'w', 'T', ' ', '8', '\n', 'ú', 'b', 'É', 'R'}


In [145]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters) # 69
num_decoder_tokens = len(target_characters) # 84
max_encoder_seq_length = max([len(txt) for txt in input_texts]) #16
max_decoder_seq_length = max([len(txt) for txt in target_texts]) # 78

print("No.of samples:", len(input_texts))
print("No.of unique input tokens:", num_encoder_tokens)
print("No.of unique output tokens:", num_decoder_tokens)
print("Maximum seq length for inputs:", max_encoder_seq_length)
print("Maximum seq length for outputs:", max_decoder_seq_length)

No.of samples: 10000
No.of unique input tokens: 69
No.of unique output tokens: 84
Maximum seq length for inputs: 16
Maximum seq length for outputs: 78


In [146]:
print(input_characters)

print("\n")

print(target_characters)

[' ', '!', '$', "'", ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'Y', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


['\t', '\n', ' ', '!', '"', "'", ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', ':', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'Y', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '¡', '«', '»', '¿', 'Á', 'É', 'Ó', 'Ú', 'á', 'é', 'í', 'ñ', 'ó', 'ú', 'ü']


In [147]:
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

print(input_token_index)

print("\n")

print(target_token_index)

{' ': 0, '!': 1, '$': 2, "'": 3, ',': 4, '-': 5, '.': 6, '0': 7, '1': 8, '2': 9, '3': 10, '4': 11, '5': 12, '6': 13, '7': 14, '8': 15, '9': 16, ':': 17, '?': 18, 'A': 19, 'B': 20, 'C': 21, 'D': 22, 'E': 23, 'F': 24, 'G': 25, 'H': 26, 'I': 27, 'J': 28, 'K': 29, 'L': 30, 'M': 31, 'N': 32, 'O': 33, 'P': 34, 'Q': 35, 'R': 36, 'S': 37, 'T': 38, 'U': 39, 'V': 40, 'W': 41, 'Y': 42, 'a': 43, 'b': 44, 'c': 45, 'd': 46, 'e': 47, 'f': 48, 'g': 49, 'h': 50, 'i': 51, 'j': 52, 'k': 53, 'l': 54, 'm': 55, 'n': 56, 'o': 57, 'p': 58, 'q': 59, 'r': 60, 's': 61, 't': 62, 'u': 63, 'v': 64, 'w': 65, 'x': 66, 'y': 67, 'z': 68}


{'\t': 0, '\n': 1, ' ': 2, '!': 3, '"': 4, "'": 5, ',': 6, '-': 7, '.': 8, '0': 9, '1': 10, '2': 11, '3': 12, '4': 13, '5': 14, '6': 15, '7': 16, '8': 17, ':': 18, '?': 19, 'A': 20, 'B': 21, 'C': 22, 'D': 23, 'E': 24, 'F': 25, 'G': 26, 'H': 27, 'I': 28, 'J': 29, 'L': 30, 'M': 31, 'N': 32, 'O': 33, 'P': 34, 'Q': 35, 'R': 36, 'S': 37, 'T': 38, 'U': 39, 'V': 40, 'W': 41, 'Y': 42, 'a': 4

In [148]:
encoder_input_data = np.zeros(
  (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
)

decoder_input_data = np.zeros(
  (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

decoder_target_data = np.zeros(
  (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)


In [149]:
encoder_input_data.shape # (Samples(Input_Text),Max_seq_Length_Input_text,Features)

(10000, 16, 69)

In [150]:
decoder_input_data.shape # (Samples(Target_Text),Max_seq_Length_Target_text,Features)

(10000, 78, 84)

In [151]:
decoder_target_data.shape # (Samples(Target_Text),Max_seq_Length_Target_text,Features)

(10000, 78, 84)

In [152]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
    
    for t, char in enumerate(target_text):
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0

In [120]:
import keras

encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
encoder = keras.layers.LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

encoder_states = [state_h, state_c]

In [121]:
decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))
decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

In [124]:
from keras.optimizers import Adam

model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer=Adam(lr=0.01, beta_1=0.9, beta_2=0.999, decay=0.001), loss='categorical_crossentropy', metrics=["accuracy"])

model.fit([encoder_input_data,
           decoder_input_data],
           decoder_target_data,
           batch_size=batch_size,
           epochs=epochs,validation_split=0.2,)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Train on 8000 samples, validate on 2000 samples
Epoch 1/100
8000/8000 [==============================] - 67s 8ms/step - loss: 0.8149 - accuracy: 0.8112 - val_loss: 0.5507 - val_accuracy: 0.8380
Epoch 2/100
8000/8000 [==============================] - 65s 8ms/step - loss: 0.4360 - accuracy: 0.8713 - val_loss: 0.4867 - val_accuracy: 0.8550
Epoch 3/100
8000/8000 [==============================] - 65s 8ms/step - loss: 0.3925 - accuracy: 0.8830 - val_loss: 0.5562 - val_accuracy: 0.8394
Epoch 4/100
8000/8000 [==============================] - 65s 8ms/step - loss: 0.3898 - accuracy: 0.8837 - val_loss: 0.4415 - val_accuracy: 0.8646
Epoch 5/100
8000/8000 [==============================] - 65s 8ms/step - loss: 0.3517 - accuracy: 0.8937 - val_loss: 0.4187 - val_accuracy: 0.8728
Epoch 6/100
8000/8000 [==============================] - 65s 8ms/step - loss: 0.3344 - accuracy: 0.8986 - val_loss: 0.4055 - va

In [125]:
model.save("E2S")

In [128]:
from keras.utils import plot_model
#plot_model(model, to_file='modelsummary.png', show_shapes=True, show_layer_names=True)

In [129]:
# Decode the Sentence

encoder_inputs = model.input[0]  # input_1
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output  # lstm_1
encoder_states = [state_h_enc, state_c_enc] # encoder hidden states
encoder_model = keras.Model(encoder_inputs, encoder_states)

In [131]:
# Now build the model for the decoder

decoder_inputs = model.input[1]  # input_2
decoder_state_input_h = keras.Input(shape=(latent_dim,), name="input_3")
decoder_state_input_c = keras.Input(shape=(latent_dim,), name="input_4")
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)

decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

In [133]:
# Create two reverse-lookup token indexes to decode the sequence to make it readable

reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

In [134]:
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)

    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        states_value = [h, c]
    return decoded_sentence

In [153]:
#i = np.random.choice(len(input_texts))

i = 10
input_seq = encoder_input_data[i:i+1]
translation = decode_sequence(input_seq)
print('-')
print('Input:', input_texts[i])
print('Translation:', translation)

-
Input: Who?
Translation: ¿Quién?

